# SQL Snippets (WIP)

## Fetching historical data for specific dates

Some tables in the warehouse--like those in `gtfs_schedule_type2`--capture the full
history of data, as it existed every day in the past. In order to do this, they
don't save copies of the full data every day, but log changes to the data using
a `calitp_extracted_at`, and `calitp_deleted_at` column.

In order to get the data for a given day, you need to filter to keep data where..

* `calitp_extracted_at` was earlier than or on the target date.
* `calitp_deleted_at` is later than the target date.

## A single date

In [1]:
from calitp.tables import tbl
from myst_nb import glue
from calitp import query_sql
from siuba import *
import pandas as pd
import calitp.magics

In [2]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info`
WHERE
    calitp_extracted_at >= "2021-06-01"
    AND COALESCE(calitp_deleted_at, "2099-01-01") > "2021-06-01"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,feed_contact_url,calitp_extracted_at,calitp_deleted_at,calitp_hash
0,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20210614,20211010,20210727,None,None,2021-07-31,2021-08-07,/d2Mz90sBPPd4rkoi85VlQ==
1,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20210625,20210806,20210722,None,None,2021-07-31,2021-08-07,z/tR7SafFIJ1DgfCFEQJSQ==
2,4,0,AC TRANSIT,http://www.actransit.org,en,None,20210613,20210807,S1000180,None,None,2021-06-12,2021-08-07,u4A6Jnu1mU796tzmCBu4jA==
3,360,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210804,20220801,UTC: 04-Aug-2021 20:23,support+test+victorville-ca-us@trilliumtransit...,http://support.trilliumtransit.com,2021-08-04,2021-08-07,GGuO34JXlT7NfnypJF/aGw==
4,259,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210727,20211231,UTC: 27-Jul-2021 23:36,support+test+redding-ca-us@trilliumtransit.com,http://support.trilliumtransit.com,2021-07-27,2021-08-07,7iPqyDJbkyIifyRvk3gIwA==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,368,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210914,20220228,UTC: 15-Sep-2021 00:36,support+test+westcat-ca-us@trilliumtransit.com,http://support.trilliumtransit.com,2021-09-15,2021-10-08,csmpRmDT7So+h/TYbcXGjA==
656,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20201214,20220102,20211001,None,None,2021-10-02,2021-10-08,zmkTvQsBzxtg/Xy3+NfX+Q==
657,200,0,511 SF Bay,https://511.org/open-data/transit,en,None,20210828,20221231,20210920,None,None,2021-09-21,2021-10-08,uWZlb5HFzBRKjWte0bVfvw==
658,167,1,511 SF Bay,https://511.org/open-data/transit,en,None,20210907,20220527,20210928,None,None,2021-09-28,2021-10-08,onlNwz6xVBQL9dZwXnLYEQ==


Note that `COALESCE` lets us fill in NULL deleted at values to be far in the future.
This is used because when deleted at is missing, it reflects the most recent data
(i.e. data that hasn't been deleted yet).
Because `NULL < "2021-06-01"` is `false`, we need to fill it in with a far-future date,
so it evaluates to `true`.

## Multiple dates

In order to do it for a range of dates, you can use a JOIN. This is shown below.

In [3]:
%%sql -m
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```SQL
SELECT *
FROM `gtfs_schedule_type2.feed_info` FI
JOIN `views.dim_date` D
    ON FI.calitp_extracted_at <= D.full_date
        AND COALESCE(FI.calitp_deleted_at, "2099-01-01") > D.full_date
WHERE
    D.full_date BETWEEN "2021-06-01" AND "2021-06-07"

```

,calitp_itp_id,calitp_url_number,feed_publisher_name,feed_publisher_url,feed_lang,default_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,...,month_name,week_day,day_name,day_is_weekday,is_quarter_start,is_month_start,is_in_past,is_in_past_or_present,is_in_future,is_gtfs_schedule_range
0,17,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201230,20210701,UTC: 30-Dec-2020 19:48,support+test+arcadia-ca-us@trilliumtransit.com,...,June,0,Sunday,0,False,False,True,True,False,True
1,17,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201230,20210701,UTC: 30-Dec-2020 19:48,support+test+arcadia-ca-us@trilliumtransit.com,...,June,3,Wednesday,1,False,False,True,True,False,True
2,17,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201230,20210701,UTC: 30-Dec-2020 19:48,support+test+arcadia-ca-us@trilliumtransit.com,...,June,5,Friday,1,False,False,True,True,False,True
3,17,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201230,20210701,UTC: 30-Dec-2020 19:48,support+test+arcadia-ca-us@trilliumtransit.com,...,June,6,Saturday,0,False,False,True,True,False,True
4,17,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20201230,20210701,UTC: 30-Dec-2020 19:48,support+test+arcadia-ca-us@trilliumtransit.com,...,June,1,Monday,1,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,7,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210428,20210628,UTC: 28-Apr-2021 22:31,support+test+montereypark-ca-us@trilliumtransi...,...,June,5,Friday,1,False,False,True,True,False,True
1305,7,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210428,20210628,UTC: 28-Apr-2021 22:31,support+test+montereypark-ca-us@trilliumtransi...,...,June,6,Saturday,0,False,False,True,True,False,True
1306,7,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210428,20210628,UTC: 28-Apr-2021 22:31,support+test+montereypark-ca-us@trilliumtransi...,...,June,1,Monday,1,False,False,True,True,False,True
1307,7,0,"Trillium Solutions, Inc.",http://www.trilliumtransit.com,en,None,20210428,20210628,UTC: 28-Apr-2021 22:31,support+test+montereypark-ca-us@trilliumtransi...,...,June,4,Thursday,1,False,False,True,True,False,True
